# Your Title Here

**Name(s)**: Kevin Wong, Andrew Yang

**Website Link**: (your website link)

# Introduction
<span style="color:red">Understand the data you have access to. Brainstorm a few questions that interest you about the dataset. Pick one question you plan to investigate further. (As the data science lifecycle tells us, this question may change as you work on your project.)</span>

# Univariate Analysis
<span style="color:red">Look at the distributions of relevant columns separately by using DataFrame operations and drawing at least two relevant plots</span>

# Bivariate Analysis
<span style="color:red">Look at the statistics of pairs of columns to identify possible associations. For instance, you may create scatter plots and plot conditional distributions, or box-plots. You must plot at least two such plots in your notebook. The results of your bivariate analyses will be helpful in identifying interesting hypothesis tests!	</span>

# Interesting Aggregates
<span style="color:red"> Choose columns to group and pivot by and examine aggregate statistics.	</span>

# NMAR Analysis
<span style="color:red"> Recall, to determine whether data are likely NMAR, you must reason about the data generating process; you cannot conclude that data are likely NMAR solely by looking at your data. As such, there’s no code to write here (and hence, nothing to put in your notebook).	</span>

# Missingness Dependency
<span style="color:red"> Pick a column in the dataset with non-trivial missingness to analyze, and perform permutation tests to analyze the dependency of the missingness of this column on other columns. </span>

<span style="color:red">Specifically, find at least one other column that the missingness of your selected column does depend on, and at least one other column that the missingness of your selected column does not depend on. </span>

<span style="color:red"> Tip: Make sure you know the difference between the different types of missingness before approaching that section. Many students in the past have lost credit for mistaking one type of missingness for another. </span>

# Hypothesis Testing
<span style="color:red"> Clearly state a pair of hypotheses and perform a hypothesis test or permutation test that is not related to missingness. Feel free to use the “sample questions” in each of the dataset descriptions or create your own. This should be the question that is stated clearly at the top of your report.	</span>

# Notes

1) GameID has 12 rows - 10 for each of the 5 players on both teams, as well as 2 for the team summary data. To separate it, 'position' column either has the position of the player, or 'team' if it is the summary stat
2) Try to convert boolean columns  
        a) 'datacompleteness' ('complete' - True, 'partial' - False) (NOTE: 'ignore' may be considered True - need to look at further data to compensate)
        b) 'playoffs' - (1 - True, 0 - False)
        c) 'result' - (1 - True, 0 - False)

### datacompleteness - 'Partial' Notes

1) 'Partial' - all data is web scraped from LPL (China), missing a lot because site is missing said data
2) Columns 'doublekill', 'triplekill', 'quadrakill', 'pentakill', 'firstblood', drakeinformation, 'firsttower', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates', 'damagemitigatedperminute', general gold information

1) Null Hypothesis: For teams that win, the probability of having a winning top lane has no affect on how well the team does

2) Alternative Hypothesis: For teams that win, the probabilityy of having a winning top is significantly higher than when the team loses.

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Cleaning and EDA

In [10]:
df = pd.read_csv("2022_LoL_esports_match_data_from_OraclesElixir.csv")

/Users/kevinwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (2,76) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
# cleaning data: converts columns 'datacompleteness', 'playoffs', 'result'
# we then get the columns that are needed for our hypothesis

cleaned_data = df.assign(
    datacompleteness = df['datacompleteness'].apply(lambda x: False if x == "incomplete" else True),
    playoffs = df['playoffs'].apply(lambda x: True if x == 1 else False),
    result = df['result'].apply(lambda x: True if x == 1 else False)
    ).loc[
        :, ['gameid', 'datacompleteness', 'playoffs', 'side', 'position', 'champion', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'total cs', 'cspm', 'totalgold']
    ]

In [23]:
# filter the dataframe to only get complete data and position that we want to analyze
filtered_df = cleaned_data[(cleaned_data['datacompleteness'] == True) & ((cleaned_data['position'] == 'top')|(cleaned_data['position'] == 'team'))]

In [29]:
filtered_df

,gameid,datacompleteness,playoffs,side,position,champion,result,kills,deaths,assists,teamkills,teamdeaths,total cs,cspm,totalgold
0,ESPORTSTMNT01_2690210,True,False,Blue,top,Renekton,False,2,3,2,9,19,231.0,8.0911,10934
5,ESPORTSTMNT01_2690210,True,False,Red,top,Gragas,True,1,1,12,19,9,229.0,8.0210,10001
10,ESPORTSTMNT01_2690210,True,False,Blue,team,NaN,False,9,19,19,9,19,NaN,29.4221,47070
11,ESPORTSTMNT01_2690210,True,False,Red,team,NaN,True,19,9,62,19,9,NaN,34.1856,52617
12,ESPORTSTMNT01_2690219,True,False,Blue,top,Gragas,False,0,5,2,3,16,245.0,6.9536,11076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149387,9687-9687_game_4,True,False,Red,team,NaN,False,7,19,16,7,19,NaN,NaN,56573
149388,9687-9687_game_5,True,False,Blue,top,Gragas,False,0,4,2,8,21,197.0,6.6479,8594
149393,9687-9687_game_5,True,False,Red,top,Jax,True,4,0,5,21,8,281.0,9.4826,13735
149398,9687-9687_game_5,True,False,Blue,team,NaN,False,8,21,16,8,21,NaN,NaN,45976


### Assessment of Missingness

In [ ]:
# TODO

### Hypothesis Testing

In [ ]:
# TODO